In [1]:
import os, glob, json, csv, subprocess, sys, re
from git import *
from subprocess import Popen, PIPE
from os import path
import pandas as pd
import plotly.graph_objs as go
import plotly
from plotly.offline import plot, init_notebook_mode, iplot
import itertools

%matplotlib inline

# Defining repository

In [2]:
userhome = os.path.expanduser('~')

In [3]:
repository = userhome + r'/historage-mining/historage/hbase/'

# Defining git command function

In [4]:
def execute_command(cmd, work_dir):
    """Executes a shell command in a subprocess, waiting until it has completed.
 
    :param cmd: Command to execute.
    :param work_dir: Working directory path.
    """
    pipe = subprocess.Popen(cmd, shell=True, cwd=work_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    (out, error) = pipe.communicate()
    return out, error
    pipe.wait()

In [5]:
def execute_shell_command(buglink, id1, id2, cmd, work_dir):
    """Executes a shell command in a subprocess, waiting until it has completed.
 
    :param cmd: Command to execute.
    :param work_dir: Working directory path.
    """
    pipe = subprocess.Popen(cmd, shell=True, cwd=work_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    (out, error) = pipe.communicate()
    return buglink, id1, id2, out, error
    pipe.wait()

# Finding log messages containing bug link

In [6]:
git_cmd = "git log --grep='bug' --oneline"
com_log = (str(execute_command(git_cmd, repository)).replace("b'",'').replace('(','',1).split("\\n"))[:-1]

com_log

['07e93458fe HBASE-19930: Fixing the bug, in the rare case when there is a merge into CCM and the one of the old segments has a big cell allocated on-heap and it needs to be copied to the MSLAB',
 '001f9cc5ea     HBASE-19940 TestMetaShutdownHandler flakey; ADDENDUM: yet more debug',
 '590bee78dc HBASE-19940 TestMetaShutdownHandler flakey; ADDENDUM: more debug',
 '9f2149f171 HBASE-19840 Flakey TestMetaWithReplicas; ADDENDUM Adding debug',
 'c245bd5c03 HBASE-19837 Flakey TestRegionLoad; ADDENDUM Report more often and wait less time on change (also add some debug on TestMetaShutdown test)',
 '88d6e06a1f HBASE-19554 reenable TestDLSAsyncFSWAL/TestDLSFSHLog for debugging',
 '811afad103 HBASE-19915 Create merged/ daughter region/s with initial state CLOSED from MetaTableAccessor#splitRegion()/ mergeRegions()',
 '34c6c99041 HBASE-19811 Fix findbugs and error-prone warnings in hbase-server (branch-2) - addendum',
 'aeffca497b HBASE-19846 Fix findbugs and error-prone warnings in hbase-rest (bra

In [7]:
commit = []
for xx in range(0,len(com_log)):
    tmp = []
    comm = com_log[xx].split()
    word = ' '.join(comm[1:])
    tmp.extend([comm[0],word])
    commit.append(tmp)

for item in commit:
    print (item)

['07e93458fe', 'HBASE-19930: Fixing the bug, in the rare case when there is a merge into CCM and the one of the old segments has a big cell allocated on-heap and it needs to be copied to the MSLAB']
['001f9cc5ea', 'HBASE-19940 TestMetaShutdownHandler flakey; ADDENDUM: yet more debug']
['590bee78dc', 'HBASE-19940 TestMetaShutdownHandler flakey; ADDENDUM: more debug']
['9f2149f171', 'HBASE-19840 Flakey TestMetaWithReplicas; ADDENDUM Adding debug']
['c245bd5c03', 'HBASE-19837 Flakey TestRegionLoad; ADDENDUM Report more often and wait less time on change (also add some debug on TestMetaShutdown test)']
['88d6e06a1f', 'HBASE-19554 reenable TestDLSAsyncFSWAL/TestDLSFSHLog for debugging']
['811afad103', 'HBASE-19915 Create merged/ daughter region/s with initial state CLOSED from MetaTableAccessor#splitRegion()/ mergeRegions()']
['34c6c99041', 'HBASE-19811 Fix findbugs and error-prone warnings in hbase-server (branch-2) - addendum']
['aeffca497b', 'HBASE-19846 Fix findbugs and error-prone warn

In [8]:
#commitbugs = []
bug_links = []

for xx in range(0,len(com_log)):
    tmp = []
    comm = com_log[xx].split()
    splitres = (str(comm[int(len(comm)/2):]))
    letter = re.findall(r"[HBASE]+[-][0-9]+", splitres)
    if letter != []:
        bug_links.extend(letter)
        #word = ' '.join(comm[1:len(comm)])
        #tmp.extend([comm[0],word])
        #commitbugs.append(tmp)
    else:
        pass

bug_links = sorted(bug_links, reverse=True)

In [9]:
with open ("bug_links.txt", mode="wt", encoding="utf-8") as myfile:
    myfile.write('\n'.join(bug_links))
print ("File bug_links.txt has been created")

File bug_links.txt has been created


git log --oneline | grep 13686

In [10]:
com_log = []
for no, bug_link in enumerate(bug_links):
    git_cmd = "git log --grep='" + bug_link + "' --oneline"
    temp = (str(execute_command(git_cmd, repository)).replace("b'",'').replace('(','',1).split("\\n"))[:-1]

    com_log.append(temp)

for item in com_log:
    print(item)

['b5d161c513 HBASE-10397 Fix findbugs introduced from HBASE-9426.', 'c079ba4660 HBASE-9426 Make custom distributed barrier procedure pluggable (Richard Ding)']
['3fd6997380 HBASE-5596 Few minor bugs from HBASE-5209 (David S. Wang)', '145d25dfc2 HBASE-5209 HConnection/HMasterInterface should allow for way to get hostname of currently active master in multi-master HBase setup']
['4421fdf981 HBASE-16066 Resolve RpC_REPEATED_CONDITIONAL_TEST findbugs warnings in HMaster, addendum to HBASE-15467', 'bdb0cc8808 HBASE-15467 Remove 1.x/2.0 TableDescriptor incompatibility']
['84a50393ee HBASE-13888 Fix refill bug from HBASE-13686 (Guanghao Zhang)', '9f43a3bea6 HBASE-13686 - Fail to limit rate in RateLimiter (Ashish Singhi)']
['bc9834426a HBASE-10624 Fix 2 new findbugs warnings introduced by HBASE-10598', 'c6236a57a5 HBASE-10598 Written data can not be read out because MemStore#timeRangeTracker might be updated concurrently (cuijianwei)']
['3b56d2a0bc HBASE-12956 Binding to 0.0.0.0 is broken afte

In [11]:
commits = []
for x in range(0,len(com_log)):
    com = []
    com.append(bug_links[x])
    for z in range(0,len(com_log[x])):
        temp = str(com_log[x][z].split()[0:1]).replace("['",'').replace("']",'')
        com.append(temp)
    commits.append(com)
commits

[['HBASE-9426', 'b5d161c513', 'c079ba4660'],
 ['HBASE-5209', '3fd6997380', '145d25dfc2'],
 ['HBASE-15467', '4421fdf981', 'bdb0cc8808'],
 ['HBASE-13686', '84a50393ee', '9f43a3bea6'],
 ['HBASE-10598', 'bc9834426a', 'c6236a57a5'],
 ['HBASE-10569', '3b56d2a0bc', '4487333e5e', 'd59d9b48f4', '7716a3ca94']]

#git log --pretty=%P -n1 145d25dfc2
#git diff d69299dfdc44a09c7ac2a6a2c4bfffae4a49a327 145d25dfc2
#git diff --name-status d69299dfdc44a09c7ac2a6a2c4bfffae4a49a327 145d25dfc2

In [12]:
commitid = []
for x, y in enumerate(commits):
    commitid.append(commits[x][2:])
    
commitid

[['c079ba4660'],
 ['145d25dfc2'],
 ['bdb0cc8808'],
 ['9f43a3bea6'],
 ['c6236a57a5'],
 ['4487333e5e', 'd59d9b48f4', '7716a3ca94']]

# Finding parenthash

In [13]:
parenthash = []
for v, w in enumerate(commitid):
    bl = []
    bl.append(bug_links[v])
    for x, y in enumerate(commitid[v]):
        bl.append(commitid[v][x])
        git_cmd = "git log --pretty=%P -n1 " + commitid[v][x]
        temp = (str(execute_command(git_cmd, repository)).replace("b'",'').replace('(','',1).split("\\n"))[:-1]
        bl.append(temp[0])
    parenthash.append(bl)
    
parenthash

[['HBASE-9426', 'c079ba4660', 'c7a72479b6f06a1ab6de66d5d95257f87bb44500'],
 ['HBASE-5209', '145d25dfc2', 'd69299dfdc44a09c7ac2a6a2c4bfffae4a49a327'],
 ['HBASE-15467', 'bdb0cc8808', '65a8d77433cf72ce67e8b5e2efb35f2c2603c349'],
 ['HBASE-13686', '9f43a3bea6', '587b0b4f20bdc0415b6541023e611b69c87dba15'],
 ['HBASE-10598', 'c6236a57a5', 'e09d637d7a61a6e636905fa19495fb145c548d81'],
 ['HBASE-10569',
  '4487333e5e',
  'e44d0d8083e5d4087f1c3a32058daa779c1d3fbb',
  'd59d9b48f4',
  '7716a3ca949670f0a88472cbb07e2c63e5a50c35',
  '7716a3ca94',
  '0e5d21e96ae24e8c658a1b74843b72bddcb7435b']]

In [14]:
len(parenthash[5])

7

In [15]:
with open("parenthash.csv", 'w') as parent:
    header = ['bug_link','commitID','parenthash']
    writers = csv.writer(parent, delimiter=',')
    writers.writerow(header)
    for k in range(0,len(parenthash)):
        if len(parenthash[k]) <= 3:
            writers.writerow(parenthash[k])
        else:
            x = 1
            y = 2
            while y <= len(parenthash[k]):
                temp = []
                temp.append(parenthash[k][0])
                for col2 in (x,y):
                    temp.append(parenthash[k][col2])
                writers.writerow(temp)
                x = x + 2
                y = y + 2

# Applying git diff without any algorithms

In [16]:
diffs = []
for v in range(0,len(parenthash)):
    p1 = 2
    p2 = 1
    diff = []
    while p1 < len(parenthash[v]):
        diff_cmd = "git diff " + parenthash[v][p1] + " " + parenthash[v][p2] + " --stat"
        tmp = (str(execute_shell_command(parenthash[v][0],parenthash[v][p1] , parenthash[v][p2], diff_cmd, repository)).replace("(",'').
               replace("+",'').replace("-",'').replace("\\n",'').replace("b'",'').replace("(+)",'').replace("(-)",'').
               replace(")",'').replace("|",'').replace(" => ",'=>').replace("... ",'...').replace("...> ",'...>').
               replace("...=> ",'...=>').replace(" Code ","Code"))
        tmp = re.sub("([a-z])" and "([ ])", r"\1,", tmp)
        tmp2 = (str(tmp).replace("\'",'').replace(", ",'').replace(",files",'files').replace(",changed","changed").
                replace(",file",'files').replace(",insertions",'insertions').replace(",insertion",'insertions').
                replace(",deletions,",'deletions').replace(",deletion,",'deletions').replace(" ",'').
                replace("HBASE",'HBASE-'))
        diff.append(str(tmp2))
        p1 = p1 + 2
        p2 = p2 + 2
    diffs.append(diff)
    
diffresult = []
for item in diffs:
    if item == ', ':
        del item
    else:
        diffresult.append(item)
#diffresult[5]

In [17]:
diffres = []
for x in range(0,len(diffresult)):
    for y in range(0,len(diffresult[x])):
        line = diffresult[x][y]
        if "changed" not in diffresult[x][y]:
            pass
        else:
            if "deletions" not in diffresult[x][y]:
                diffres.append(line + '0 deletions')
            else:
                if "insertions" not in diffresult[x][y]:
                    index = line.find('changed,')
                    diffres.append(line[:index+8] + '0 insertions' + line[index+7:])
                else:
                    diffres.append(diffresult[x][y])
#diffres

In [18]:
kl = []
res = []
for v in range(0,len(diffres)):
    kl = diffres[v].split(',')
    res.append(kl)
    del res[v][-3:]
#res

In [19]:
with open('diff-bugs.csv', 'w') as csvfile:
    header = ['bug_ref','commitID_1', 'commitID_2', 'filename', '#line_changed']
    writers = csv.writer(csvfile, delimiter=',')
    writers.writerow(header)
    for m in range(0,len(res)):
        x = 3
        y = 5
        if len(res[m])<=5:
            writers.writerow(res[m])
        else:
            s = (len(res[m]))/2
            for g in range(1, int(s)):
                v = []
                for col1 in range(0,3):
                    v.append(res[m][col1])
                for col2 in range(x,y):
                    if re.findall(r'\d+',res[m][col2]) == True:
                        v.append(re.findall(r'\d+',res[m][col2]))
                    else:
                        v.append(res[m][col2])
                x = x + 2
                y = y + 2
                writers.writerow(v)

# Applying git diff -w

In [20]:
diffsw = []
for v in range(0,len(parenthash)):
    p1 = 2
    p2 = 1
    diff = []
    while p1 < len(parenthash[v]):
        diff_cmd = "git diff -w " + parenthash[v][p1] + " " + parenthash[v][p2] + " --stat"
        tmp = (str(execute_shell_command(parenthash[v][0],parenthash[v][p1] , parenthash[v][p2], diff_cmd, repository)).replace("(",'').
               replace("+",'').replace("-",'').replace("\\n",'').replace("b'",'').replace("(+)",'').replace("(-)",'').
               replace(")",'').replace("|",'').replace(" => ",'=>').replace("... ",'...').replace("...> ",'...>').
               replace("...=> ",'...=>').replace(" Code ","Code"))
        tmp = re.sub("([a-z])" and "([ ])", r"\1,", tmp)
        tmp2 = (str(tmp).replace("\'",'').replace(", ",'').replace(",files",'files').replace(",changed","changed").
                replace(",file",'files').replace(",insertions",'insertions').replace(",insertion",'insertions').
                replace(",deletions,",'deletions').replace(",deletion,",'deletions').replace(" ",'').
                replace("HBASE",'HBASE-'))
        diff.append(str(tmp2))
        p1 = p1 + 2
        p2 = p2 + 2
    diffsw.append(diff)
    
diffwresult = []
for item in diffsw:
    if item == ', ':
        del item
    else:
        diffwresult.append(item)
#diffwresult[0]

In [21]:
diffwres = []
for x in range(0,len(diffwresult)):
    for y in range(0,len(diffwresult[x])):
        line = diffwresult[x][y]
        if "changed" not in diffwresult[x][y]:
            pass
        else:
            if "deletions" not in diffwresult[x][y]:
                diffwres.append(line + '0 deletions')
            else:
                if "insertions" not in diffwresult[x][y]:
                    index = line.find('changed,')
                    diffwres.append(line[:index+8] + '0 insertions' + line[index+7:])
                else:
                    diffwres.append(diffwresult[x][y])

In [22]:
klw = []
resw = []
for x in range(0, len(diffwres)):
    klw = diffwres[x].split(',')
    resw.append(klw)
    del resw[x][-3:]
#resw

In [23]:
with open('diffw-bugs.csv', 'w') as csvfile:
    header = ['bug_ref','commitID_1', 'commitID_2', 'filename', '#line_changed']
    writers = csv.writer(csvfile, delimiter=',')
    writers.writerow(header)
    for m in range(0,len(resw)):
        x = 3
        y = 5
        if len(resw[m])<=5:
            writers.writerow(resw[m])
        else:
            s = (len(resw[m]))/2
            for g in range(1, int(s)):
                v = []
                for col1 in range(0,3):
                    v.append(resw[m][col1])
                for col2 in range(x,y):
                    if re.findall(r'\d+',resw[m][col2]) == True:
                        v.append(re.findall(r'\d+',resw[m][col2]))
                    else:
                        v.append(resw[m][col2])
                x = x + 2
                y = y + 2
                writers.writerow(v)

# Applying git diff -w --diff-algorithm=myers

In [24]:
diffmyers = []
for v in range(0,len(parenthash)):
    p1 = 2
    p2 = 1
    diff = []
    while p1 < len(parenthash[v]):
        diff_cmd = "git diff -w --diff-algorithm=myers " + parenthash[v][p1] + " " + parenthash[v][p2] + " --stat"
        tmp = (str(execute_shell_command(parenthash[v][0],parenthash[v][p1] , parenthash[v][p2], diff_cmd, repository)).replace("(",'').
               replace("+",'').replace("-",'').replace("\\n",'').replace("b'",'').replace("(+)",'').replace("(-)",'').
               replace(")",'').replace("|",'').replace(" => ",'=>').replace("... ",'...').replace("...> ",'...>').
               replace("...=> ",'...=>').replace(" Code ","Code"))
        tmp = re.sub("([a-z])" and "([ ])", r"\1,", tmp)
        tmp2 = (str(tmp).replace("\'",'').replace(", ",'').replace(",files",'files').replace(",changed","changed").
                replace(",file",'files').replace(",insertions",'insertions').replace(",insertion",'insertions').
                replace(",deletions,",'deletions').replace(",deletion,",'deletions').replace(" ",'').
                replace("HBASE",'HBASE-'))
        diff.append(str(tmp2))
        p1 = p1 + 2
        p2 = p2 + 2
    diffmyers.append(diff)
    
diffmyersresult = []
for item in diffmyers:
    if item == ', ':
        del item
    else:
        diffmyersresult.append(item)

In [25]:
diffmyersres = []
for x in range(0,len(diffmyersresult)):
    for y in range(0,len(diffmyersresult[x])):
        line = diffmyersresult[x][y]
        if "changed" not in diffmyersresult[x][y]:
            pass
        else:
            if "deletions" not in diffmyersresult[x][y]:
                diffmyersres.append(line + '0 deletions')
            else:
                if "insertions" not in diffmyersresult[x][y]:
                    index = line.find('changed,')
                    diffmyersres.append(line[:index+8] + '0 insertions' + line[index+7:])
                else:
                    diffmyersres.append(diffmyersresult[x][y])

In [26]:
klmyers = []
resmyers = []
for x in range(0, len(diffmyersres)):
    klmyers = diffmyersres[x].split(',')
    resmyers.append(klmyers)
    del resmyers[x][-3:]

In [27]:
with open('diffmyers-bugs.csv', 'w') as csvfile:
    header = ['bug_ref','commitID_1', 'commitID_2', 'filename', '#line_changed']
    writers = csv.writer(csvfile, delimiter=',')
    writers.writerow(header)
    for m in range(0,len(resmyers)):
        x = 3
        y = 5
        if len(resmyers[m])<=5:
            writers.writerow(resmyers[m])
        else:
            s = (len(resmyers[m]))/2
            for g in range(1, int(s)):
                v = []
                for col1 in range(0,3):
                    v.append(resmyers[m][col1])
                for col2 in range(x,y):
                    if re.findall(r'\d+',resmyers[m][col2]) == True:
                        v.append(re.findall(r'\d+',resmyers[m][col2]))
                    else:
                        v.append(resmyers[m][col2])
                x = x + 2
                y = y + 2
                writers.writerow(v)

# Applying --diff-algorithm=minimal

In [28]:
diffmin = []
for v in range(0,len(parenthash)):
    p1 = 2
    p2 = 1
    diff = []
    while p1 < len(parenthash[v]):
        diff_cmd = "git diff -w --diff-algorithm=minimal " + parenthash[v][p1] + " " + parenthash[v][p2] + " --stat"
        tmp = (str(execute_shell_command(parenthash[v][0],parenthash[v][p1] , parenthash[v][p2], diff_cmd, repository)).replace("(",'').
               replace("+",'').replace("-",'').replace("\\n",'').replace("b'",'').replace("(+)",'').replace("(-)",'').
               replace(")",'').replace("|",'').replace(" => ",'=>').replace("... ",'...').replace("...> ",'...>').
               replace("...=> ",'...=>').replace(" Code ","Code"))
        tmp = re.sub("([a-z])" and "([ ])", r"\1,", tmp)
        tmp2 = (str(tmp).replace("\'",'').replace(", ",'').replace(",files",'files').replace(",changed","changed").
                replace(",file",'files').replace(",insertions",'insertions').replace(",insertion",'insertions').
                replace(",deletions,",'deletions').replace(",deletion,",'deletions').replace(" ",'').
                replace("HBASE",'HBASE-'))
        diff.append(str(tmp2))
        p1 = p1 + 2
        p2 = p2 + 2
    diffmin.append(diff)
    
diffminresult = []
for item in diffmin:
    if item == ', ':
        del item
    else:
        diffminresult.append(item)

In [29]:
diffminres = []
for x in range(0,len(diffminresult)):
    for y in range(0,len(diffminresult[x])):
        line = diffminresult[x][y]
        if "changed" not in diffminresult[x][y]:
            pass
        else:
            if "deletions" not in diffminresult[x][y]:
                diffminres.append(line + '0 deletions')
            else:
                if "insertions" not in diffminresult[x][y]:
                    index = line.find('changed,')
                    diffminres.append(line[:index+8] + '0 insertions' + line[index+7:])
                else:
                    diffminres.append(diffminresult[x][y])

In [30]:
klmin = []
resmin = []
for x in range(0, len(diffminres)):
    klmin = diffminres[x].split(',')
    resmin.append(klmin)
    del resmin[x][-3:]

In [31]:
with open('diffmin-bugs.csv', 'w') as csvfile:
    header = ['bug_ref','commitID_1', 'commitID_2', 'filename', '#line_changed']
    writers = csv.writer(csvfile, delimiter=',')
    writers.writerow(header)
    for m in range(0,len(resmin)):
        x = 3
        y = 5
        if len(resmin[m])<=5:
            writers.writerow(resmin[m])
        else:
            s = (len(resmin[m]))/2
            for g in range(1, int(s)):
                v = []
                for col1 in range(0,3):
                    v.append(resmin[m][col1])
                for col2 in range(x,y):
                    if re.findall(r'\d+',resmin[m][col2]) == True:
                        v.append(re.findall(r'\d+',resmin[m][col2]))
                    else:
                        v.append(resmin[m][col2])
                x = x + 2
                y = y + 2
                writers.writerow(v)

# Applying --diff-algorithm=patience

In [32]:
diffpat = []
for v in range(0,len(parenthash)):
    p1 = 2
    p2 = 1
    diff = []
    while p1 < len(parenthash[v]):
        diff_cmd = "git diff -w --diff-algorithm=patience " + parenthash[v][p1] + " " + parenthash[v][p2] + " --stat"
        tmp = (str(execute_shell_command(parenthash[v][0],parenthash[v][p1] , parenthash[v][p2], diff_cmd, repository)).replace("(",'').
               replace("+",'').replace("-",'').replace("\\n",'').replace("b'",'').replace("(+)",'').replace("(-)",'').
               replace(")",'').replace("|",'').replace(" => ",'=>').replace("... ",'...').replace("...> ",'...>').
               replace("...=> ",'...=>').replace(" Code ","Code"))
        tmp = re.sub("([a-z])" and "([ ])", r"\1,", tmp)
        tmp2 = (str(tmp).replace("\'",'').replace(", ",'').replace(",files",'files').replace(",changed","changed").
                replace(",file",'files').replace(",insertions",'insertions').replace(",insertion",'insertions').
                replace(",deletions,",'deletions').replace(",deletion,",'deletions').replace(" ",'').
                replace("HBASE",'HBASE-'))
        diff.append(str(tmp2))
        p1 = p1 + 2
        p2 = p2 + 2
    diffpat.append(diff)
    
diffpatresult = []
for item in diffpat:
    if item == ', ':
        del item
    else:
        diffpatresult.append(item)

In [33]:
diffpatres = []
for x in range(0,len(diffpatresult)):
    for y in range(0,len(diffpatresult[x])):
        line = diffpatresult[x][y]
        if "changed" not in diffpatresult[x][y]:
            pass
        else:
            if "deletions" not in diffpatresult[x][y]:
                diffpatres.append(line + '0 deletions')
            else:
                if "insertions" not in diffpatresult[x][y]:
                    index = line.find('changed,')
                    diffpatres.append(line[:index+8] + '0 insertions' + line[index+7:])
                else:
                    diffpatres.append(diffpatresult[x][y])

In [34]:
klpat = []
respat = []
for x in range(0, len(diffpatres)):
    klpat = diffpatres[x].split(',')
    respat.append(klpat)
    del respat[x][-3:]

In [35]:
with open('diffpat-bugs.csv', 'w') as csvfile:
    header = ['bug_ref','commitID_1', 'commitID_2', 'filename', '#line_changed']
    writers = csv.writer(csvfile, delimiter=',')
    writers.writerow(header)
    for m in range(0,len(respat)):
        x = 3
        y = 5
        if len(respat[m])<=5:
            writers.writerow(respat[m])
        else:
            s = (len(respat[m]))/2
            for g in range(1, int(s)):
                v = []
                for col1 in range(0,3):
                    v.append(respat[m][col1])
                for col2 in range(x,y):
                    if re.findall(r'\d+',respat[m][col2]) == True:
                        v.append(re.findall(r'\d+',respat[m][col2]))
                    else:
                        v.append(respat[m][col2])
                x = x + 2
                y = y + 2
                writers.writerow(v)

# Applying --diff-algorithm=histogram

In [36]:
diffhist = []
for v in range(0,len(parenthash)):
    p1 = 2
    p2 = 1
    diff = []
    while p1 < len(parenthash[v]):
        diff_cmd = "git diff -w --diff-algorithm=histogram " + parenthash[v][p1] + " " + parenthash[v][p2] + " --stat"
        tmp = (str(execute_shell_command(parenthash[v][0],parenthash[v][p1] , parenthash[v][p2], diff_cmd, repository)).replace("(",'').
               replace("+",'').replace("-",'').replace("\\n",'').replace("b'",'').replace("(+)",'').replace("(-)",'').
               replace(")",'').replace("|",'').replace(" => ",'=>').replace("... ",'...').replace("...> ",'...>').
               replace("...=> ",'...=>').replace(" Code ","Code"))
        tmp = re.sub("([a-z])" and "([ ])", r"\1,", tmp)
        tmp2 = (str(tmp).replace("\'",'').replace(", ",'').replace(",files",'files').replace(",changed","changed").
                replace(",file",'files').replace(",insertions",'insertions').replace(",insertion",'insertions').
                replace(",deletions,",'deletions').replace(",deletion,",'deletions').replace(" ",'').
                replace("HBASE",'HBASE-'))
        diff.append(str(tmp2))
        p1 = p1 + 2
        p2 = p2 + 2
    diffhist.append(diff)
    
diffhistresult = []
for item in diffhist:
    if item == ', ':
        del item
    else:
        diffhistresult.append(item)

In [37]:
diffhistres = []
for x in range(0,len(diffhistresult)):
    for y in range(0,len(diffhistresult[x])):
        line = diffhistresult[x][y]
        if "changed" not in diffhistresult[x][y]:
            pass
        else:
            if "deletions" not in diffhistresult[x][y]:
                diffhistres.append(line + '0 deletions')
            else:
                if "insertions" not in diffpatresult[x][y]:
                    index = line.find('changed,')
                    diffhistres.append(line[:index+8] + '0 insertions' + line[index+7:])
                else:
                    diffhistres.append(diffhistresult[x][y])

In [38]:
klhist = []
reshist = []
for x in range(0, len(diffhistres)):
    klhist = diffhistres[x].split(',')
    reshist.append(klhist)
    del reshist[x][-3:]

In [39]:
with open('diffhist-bugs.csv', 'w') as csvfile:
    header = ['bug_ref','commitID_1', 'commitID_2', 'filename', '#line_changed']
    writers = csv.writer(csvfile, delimiter=',')
    writers.writerow(header)
    for m in range(0,len(reshist)):
        x = 3
        y = 5
        if len(reshist[m])<=5:
            writers.writerow(reshist[m])
        else:
            s = (len(reshist[m]))/2
            for g in range(1, int(s)):
                v = []
                for col1 in range(0,3):
                    v.append(reshist[m][col1])
                for col2 in range(x,y):
                    if re.findall(r'\d+',reshist[m][col2]) == True:
                        v.append(re.findall(r'\d+',reshist[m][col2]))
                    else:
                        v.append(reshist[m][col2])
                x = x + 2
                y = y + 2
                writers.writerow(v)

# Visualizing the number of changes in different diff using graph

In [40]:
col = ['filename','#line_changed']
df = pd.read_csv('diff-bugs.csv')
dfw = pd.read_csv('diffw-bugs.csv')
dfmyers = pd.read_csv('diffmyers-bugs.csv')
dfhist = pd.read_csv('diffhist-bugs.csv')
dfmin = pd.read_csv('diffmin-bugs.csv')
dfpat = pd.read_csv('diffpat-bugs.csv')

df = df.sort_values('#line_changed', ascending=False)
dfw = dfw.sort_values('#line_changed', ascending=False)
dfmyers = dfmyers.sort_values('#line_changed', ascending=False)
dfhist = dfhist.sort_values('#line_changed', ascending=False)
dfmin = dfmin.sort_values('#line_changed', ascending=False)
dfpat = dfpat.sort_values('#line_changed', ascending=False)

trace = go.Bar(
    x=df['filename'][:10],
    y=df['#line_changed'][:10],
    name='no diff-algorithm')

tracew = go.Bar(
    x=dfw['filename'][:10],
    y=dfw['#line_changed'][:10],
    name='adding -w in diff')

tracemyers = go.Bar(
    x=dfmyers['filename'][:10],
    y=dfmyers['#line_changed'][:10],
    name='--diff-algorithm=myers')

tracehist = go.Bar(
    x=dfhist['filename'][:10],
    y=dfhist['#line_changed'][:10],
    name='--diff-algorithm=histogram')

tracemin = go.Bar(
    x=dfmin['filename'][:10],
    y=dfmin['#line_changed'][:10],
    name='--diff-algorithm=minimal')

tracepat = go.Bar(
    x=dfpat['filename'][:10],
    y=dfpat['#line_changed'][:10],
    name='--diff-algorithm=patience')

dt = [tracemyers, tracehist, tracemin, tracepat]
layout = go.Layout(
    barmode='group'
)

init_notebook_mode(connected=True)
fig = go.Figure(data=dt, layout=layout)
iplot(fig, show_link=False)

In [41]:
#sorting data based on filename
dfhist = dfhist.sort_values('filename')
dfmyers = dfmyers.sort_values('filename')
dfmin = dfmin.sort_values('filename')
dfpat = dfpat.sort_values('filename')

In [42]:
#Merge dataframes bug-hist and bug-myers
datamerge1 = dfhist.merge(dfmyers, on=['bug_ref','commitID_1', 'commitID_2', 'filename'], how="outer", suffixes=('-hist', '-myers'))
#Merge dataframes bug-min and bug-pat
datamerge2 = dfmin.merge(dfpat, on=['bug_ref','commitID_1', 'commitID_2', 'filename'], how="outer", suffixes=('-min', '-pat'))
#Merge all dataframes
datamerge3 = datamerge1.merge(datamerge2, on=['bug_ref','commitID_1', 'commitID_2', 'filename'], how="outer")

#Fill NAs with 0 and sort by filename
datamerge3.fillna(0, inplace=True)
datamerge3 = datamerge3.sort_values(by ='#line_changed-hist').reset_index(drop=True)
colnames = ['bug_ref','commitID_1','commitID_2','filename','#line_changed-hist','#line_changed-myers','#line_changed-min','#line_changed-pat']
datamerge3 = datamerge3[colnames]

#save to csv file
datamerge3.to_csv('diffbugs_file.csv')

In [43]:
#Remove duplicates data
datamerge4 = datamerge3[datamerge3.iloc[:,-4:].diff(axis=1).fillna(0).any(1)]
datamerge4 = datamerge4.sort_values('#line_changed-hist', ascending=True)

#save to csv file
datamerge4.to_csv('diffbugs_file_differentvalue.csv')

In [46]:
#Show the csv file with different value of number of line changed
data = pd.read_csv('diffbugs_file_differentvalue.csv')
data = datamerge4[colnames]
data

,bug_ref,commitID_1,commitID_2,filename,#line_changed-hist,#line_changed-myers,#line_changed-min,#line_changed-pat
122,HBASE-10569,0e5d21e96ae24e8c658a1b74843b72bddcb7435,7716a3ca94,.../hadoop/hbase/master/ActiveMasterManager.java,31,29,29,31
154,HBASE-13686,587b0b4f20bdc0415b6541023e611b69c87dba15,9f43a3bea6,.../apache/hadoop/hbase/quotas/RateLimiter.java,103,97,97,99
157,HBASE-13686,587b0b4f20bdc0415b6541023e611b69c87dba15,9f43a3bea6,.../hadoop/hbase/quotas/TestRateLimiter.java,114,108,108,108
172,HBASE-10569,0e5d21e96ae24e8c658a1b74843b72bddcb7435,7716a3ca94,.../org/apache/hadoop/hbase/master/HMaster.java,1993,1977,1969,1985
173,HBASE-10569,0e5d21e96ae24e8c658a1b74843b72bddcb7435,7716a3ca94,.../hadoop/hbase/regionserver/HRegionServer.java,2285,2425,2285,2285


In [47]:
tracehist = go.Bar(
    x=data['filename'],
    y=data['#line_changed-hist'],
    name='--diff-algorithm=histogram')

tracemyers = go.Bar(
    x=data['filename'],
    y=data['#line_changed-myers'],
    name='--diff-algorithm=myers')

tracemin = go.Bar(
    x=data['filename'],
    y=data['#line_changed-min'],
    name='--diff-algorithm=minimal')

tracepat = go.Bar(
    x=data['filename'],
    y=data['#line_changed-pat'],
    name='--diff-algorithm=patience')

dt = [tracemyers, tracehist, tracemin, tracepat]
layout = go.Layout(
    barmode='group'
)

init_notebook_mode(connected=True)
fig = go.Figure(data=dt, layout=layout)
iplot(fig, show_link=False)